# Preparation

In [ ]:
import re
import os
import time

import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
file_name = 'source_code.txt'
file_url = 'https://github.com/Pittawat2542/dart-ml-autocomplete/raw/master/dataset_collector/source_code.txt'
r = requests.get(file_url, allow_redirects=True)

In [ ]:
open(file_name, 'wb').write(r.content)

5148815

In [ ]:
text = open(file_name, 'rb').read().decode(encoding='utf-8')
text = re.sub(r"//.*", "", text) # remove comment
text = re.sub(r"import.*", "", text) # remove import
text = re.sub(r"export.*", "", text) # remove export
text = re.sub(r"^(?:[\t ]*(?:\r?\n|\r))+", "", text) # remove blank line
text = re.sub(r"^(\s*\r\n){2,}", "\r\n", text) # replace multiple blank lines with one
text = re.sub(r"^\s*$", "", text) # remove line with only spaces
text = text.replace('\n', '').replace(';', ';\n')
print('Length of text: {} characters'.format(len(text)))

Length of text: 4130705 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

final Context posix = Context(style: Style.posix);
final Context windows = Context(style: Style.windows);
final Context url = Context(style: Style.url);
final Context context = createInternal();
Style get style => context.style;
String get current { 


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

1114 unique characters


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[68, 69, 70, 71, 72, 73, 74], [91, 92, 93]]>

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(4130705,), dtype=int64, numpy=array([73, 76, 81, ..., 30,  2, 96])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

f
i
n
a
l
 
C
o
n
t


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'f' b'i' b'n' b'a' b'l' b' ' b'C' b'o' b'n' b't' b'e' b'x' b't' b' '
 b'p' b'o' b's' b'i' b'x' b' ' b'=' b' ' b'C' b'o' b'n' b't' b'e' b'x'
 b't' b'(' b's' b't' b'y' b'l' b'e' b':' b' ' b'S' b't' b'y' b'l' b'e'
 b'.' b'p' b'o' b's' b'i' b'x' b')' b';' b'\n' b'f' b'i' b'n' b'a' b'l'
 b' ' b'C' b'o' b'n' b't' b'e' b'x' b't' b' ' b'w' b'i' b'n' b'd' b'o'
 b'w' b's' b' ' b'=' b' ' b'C' b'o' b'n' b't' b'e' b'x' b't' b'(' b's'
 b't' b'y' b'l' b'e' b':' b' ' b'S' b't' b'y' b'l' b'e' b'.' b'w' b'i'
 b'n' b'd' b'o'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'final Context posix = Context(style: Style.posix);\nfinal Context windows = Context(style: Style.windo'
b'ws);\nfinal Context url = Context(style: Style.url);\nfinal Context context = createInternal();\nStyle g'
b'et style => context.style;\nString get current {          Uri uri;\n  try {    uri = Uri.base;\n  } on E'
b'xception {    if (_current != null) return _current!;\n    rethrow;\n  }      if (uri == _currentUriBas'
b'e) return _current!;\n  _currentUriBase = uri;\n  if (Style.platform == Style.url) {    _current = uri.'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'final Context posix = Context(style: Style.posix);\nfinal Context windows = Context(style: Style.wind'
Target: b'inal Context posix = Context(style: Style.posix);\nfinal Context windows = Context(style: Style.windo'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Modeling

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 1116) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  285696    
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  1143900   
Total params: 5,367,900
Trainable params: 5,367,900
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([ 281,  699,  842,  635,  581, 1073, 1058,   73,  305,  591,  830,
        151,  995,  378, 1052,  554,  909,  642,  717,  739,  600,  547,
        162,  745, 1102,  377, 1040,  589,  439,   50,  721,  427,  265,
        462,    7,  316,  432,  684,  779,  123,  305,  264,  969, 1005,
        140,  348,  979,  999,  180,  145,  241, 1006,  611,  634,  860,
        580,  131,  416,  455,  617,  395,  520,  480,  266,  561,  828,
       1075,  608,  772,  407,  481,  896, 1083,  692,  709,  971, 1033,
        520, 1109,  467,  479,  494,   61,   82,  460,  631,  102,  700,
        799,  116,  713,  505,  151,  905,  788,  622,  686,  126,  156,
        690])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"zz',         'ZZZZ': 'ZZZZ'       },            'sq': const {        'd': 'd',         'E': 'ccc',  "

Next Char Predictions:
 b'\xd4\xbf\xe0\xb4\x95\xe1\x80\x81\xe0\xb0\xa4\xe0\xac\xbe\xe5\x9c\x9f\xe4\xb8\x8bf\xd5\xb7\xe0\xae\x85\xe0\xba\xb5\xc5\x9e\xe1\x8f\x98\xd9\xbe\xe2\x80\xa2\xe0\xac\x86\xe1\x88\x8b\xe0\xb0\xae\xe0\xb4\xb0\xe0\xb5\xbe\xe0\xae\x9e\xe0\xab\x80\xc9\x94\xe0\xb6\x9f\xec\x8b\x9c\xd9\x94\xe1\x9f\x8d\xe0\xad\x8d\xe0\xa5\x8dO\xe0\xb4\xb4\xe0\xa4\xb9\xd1\x98\xe0\xa6\xaa$\xd6\x82\xe0\xa5\x81\xe0\xb2\xbe\xe0\xb8\x96\xc3\xbb\xd5\xb7\xd1\x97\xe1\x8e\xab\xe1\x9e\x80\xc4\xa7\xd8\xa3\xe1\x8e\xbe\xe1\x8f\xa2\xce\xa7\xc5\x82\xd0\xba\xe1\x9e\x82\xe0\xae\xb2\xe0\xb0\xa1\xe1\x80\x9c\xe0\xac\xb8\xc4\x8b\xe0\xa4\xab\xe0\xa6\x9f\xe0\xae\xbf\xe0\xa4\x82\xe0\xaa\x87\xe0\xa7\x83\xd1\x99\xe0\xac\x9c\xe0\xba\xb2\xe5\xb9\xb4\xe0\xae\xaf\xe0\xb8\x84\xe0\xa4\x9c\xe0\xa7\x87\xe1\x83\x9e\xe6\x9b\x9c\xe0\xb3\x8d\xe0\xb4\xa6\xe1\x8e\xad\xe1\x9e\xbb\xe0\xaa\x87\xec\xa0\x9c\xe0\xa6\xaf\xe0\x

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 1116)  # (batch_size, sequence_length, vocab_size)
Mean loss:         7.0169463


In [ ]:
tf.exp(mean_loss).numpy()

1115.3754

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
639/639 [==============================] - 49s 74ms/step - loss: 2.3028
Epoch 2/20
639/639 [==============================] - 46s 70ms/step - loss: 0.8565
Epoch 3/20
639/639 [==============================] - 47s 72ms/step - loss: 0.5984
Epoch 4/20
639/639 [==============================] - 46s 71ms/step - loss: 0.4841
Epoch 5/20
639/639 [==============================] - 47s 72ms/step - loss: 0.4185
Epoch 6/20
639/639 [==============================] - 46s 71ms/step - loss: 0.3731
Epoch 7/20
639/639 [==============================] - 47s 71ms/step - loss: 0.3451
Epoch 8/20
639/639 [==============================] - 47s 71ms/step - loss: 0.3230
Epoch 9/20
639/639 [==============================] - 47s 71ms/step - loss: 0.3035
Epoch 10/20
639/639 [==============================] - 47s 72ms/step - loss: 0.2918
Epoch 11/20
639/639 [==============================] - 47s 72ms/step - loss: 0.2800
Epoch 12/20
639/639 [==============================] - 47s 72ms/step - loss: 0.2720
E

# Pilot Prediction

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['const'])
result = [next_char]

for n in range(30):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

const IconDataBrands(0xf8d6);
      

________________________________________________________________________________

Run time: 0.7518947124481201


In [ ]:
start = time.time()
states = None
next_char = tf.constant(['const', 'const', 'const', 'const', 'const'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor(
[b'const IconData solidThardoNest = const IconDataSolid(0xf104);\n          static const IconData grintVide = const IconDataSolid(0xf782);\n        static const IconData fistyl = const IconDataBrands(0xf16c);\n          static const IconData biandixcletPressink = const IconDataSolid(0xf2b5);\n          static const IconData shildTieg = const IconDataSolid(0xf644);\n          static const IconData anglehter = const IconDataSolid(0xf4b5);\n        static const IconData doatsbill = const IconDataBrands(0xe084);\n        static const IconData redivi = const IconDataBrands(0xf283);\n        static const IconData barcent = const IconDataBrands(0xf784);\n          static const IconData sneploa = const IconDataSolid(0xf75e);\n        static const IconData microshte = const IconDataSolid(0xf466);\n          static const IconData sortAmountFiarkRight = const IconDataSolid(0xe06a);\n        static const IconData kisz = const IconDataBrands(0xf4eb);\n        static const IconData noCHam

# Exporting Model

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
import os
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))
      
zipf = zipfile.ZipFile('model.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('one_step/', zipf)
zipf.close()

# Prediction

In [ ]:
import requests

file_name = 'rnn_model_v2.zip'
file_url = 'https://github.com/Pittawat2542/dart-ml-autocomplete/raw/master/rnn_model_v2.zip'
r = requests.get(file_url, allow_redirects=True)
open(file_name, 'wb').write(r.content)

59907559

In [ ]:
!unzip rnn_model_v2.zip

Archive:  rnn_model_v2.zip
  inflating: one_step/saved_model.pb  
  inflating: one_step/variables/variables.data-00000-of-00001  
  inflating: one_step/variables/variables.index  


In [ ]:
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['const'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

const IconData boikAlt = const IconDataSolid(0xf2c4);
          static const IconData solidWindowSlash = 
